In [ ]:
# Установка необходимых библиотек
!pip install torch --quiet
!pip install transformers --quiet
!pip install rouge-score --quiet
!pip install pandas numpy scikit-learn tqdm --quiet

In [ ]:
# Импорт библиотек
import os

import numpy as np
import pandas as pd
import torch
from tqdm import tqdm
from rouge_score import rouge_scorer

# Устройство
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Используется устройство: {DEVICE}")

Используется устройство: cuda


In [ ]:
# Предварительная обработка данных
import os
import sys

sys.path.append("src")

# Проверка наличия файлов
if not os.path.exists("src/data_utils.py"):
    raise FileNotFoundError("Файл src/data_utils.py не найден.")

input_path = "data/raw.txt"
if not os.path.exists(input_path):
    raise FileNotFoundError(f"Входной файл не найден: {input_path}")

os.makedirs("data", exist_ok=True)

# Импорт и запуск
from src.data_utils import main as preprocess_data

print("Запуск предварительной обработки данных...")
preprocess_data()
print("Предварительная обработка завершена.")

# Проверка результатов
for split in ["train", "val", "test"]:
    path = f"data/{split}.csv"
    assert os.path.exists(path), f"Файл {path} не создан."
print("✅ Все файлы данных созданы.")

Загрузка и очистка данных...


/home/ubuntu/auto_complete_model/src/data_utils.py:55: DtypeWarning: Columns (1,2,3,4,6,7,8,10,12,16,20,24,29,31) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(RAW_DATA_PATH)


Обработано 1048562 текстов.
Токенизация текстов...
Обработанный датасет сохранён в data/dataset_processed.csv
Разбиение на train/val/test...
Размер обучающей выборки: 838849
Размер валидационной выборки: 104856
Размер тестовой выборки: 104857
Разбиение завершено.


In [ ]:
# Просмотр обработанных данных
import os

print("Просмотр обработанных данных...")

# Проверка файла
if not os.path.exists("data/train.csv"):
    raise FileNotFoundError("Файл data/train.csv не найден. Выполните предварительную обработку.")

df_train = pd.read_csv("data/train.csv")

# Проверка столбца
assert "text" in df_train.columns, "В CSV отсутствует столбец 'text'"

# Очистка
df_train.dropna(subset=["text"], inplace=True)
df_train = df_train[df_train["text"].str.strip() != ""].reset_index(drop=True)

print("Примеры обучающих текстов:")
print(df_train.head())

print(f"\nРазмер обучающей выборки: {len(df_train)}")

# Статистика по длине
lengths = df_train["text"].str.split().str.len()
print(f"Средняя длина текста: {lengths.mean():.1f} токенов")
print(f"Минимальная длина: {lengths.min()} токенов")
print(f"Максимальная длина: {lengths.max()} токенов")

if lengths.min() < 2:
    print("⚠️ В выборке есть тексты с менее чем 2 токенами — они могут быть исключены при обучении.")

Примеры обучающих текстов:
                                                text  \
0  sharxgrrl o i want to know a professional nhl ...   
1  imogenheap am's good for us gmtish people i th...   
2                         can i not go to work today   
3  hit my head on my guitar as i was taking it of...   
4  back home been interrogated for two hours they...   

                                              tokens  
0  ['sharxgrrl', 'o', 'i', 'want', 'to', 'know', ...  
1  ['imogenheap', "am's", 'good', 'for', 'us', 'g...  
2   ['can', 'i', 'not', 'go', 'to', 'work', 'today']  
3  ['hit', 'my', 'head', 'on', 'my', 'guitar', 'a...  
4  ['back', 'home', 'been', 'interrogated', 'for'...  

Размер обучающей выборки: 838849


In [ ]:
# Запуск обучения LSTM
import os

print("Запуск обучения LSTM...")

# Проверка данных
required_files = ["data/train.csv", "data/val.csv"]
for f in required_files:
    if not os.path.exists(f):
        raise FileNotFoundError(f"Файл не найден: {f}. Выполните предварительную обработку.")

# Создание папки для моделей
os.makedirs("models", exist_ok=True)

# Импорт
from src.lstm_train import train_model

# Запуск с параметрами (если функция их поддерживает)
try:
    train_model(
        num_epochs=15,
        batch_size=64,
        lr=1e-3,
        hidden_size=128,
        embedding_dim=128,
        max_length=50,
        device=DEVICE
    )
    print("✅ Обучение LSTM завершено.")
except Exception as e:
    print(f"❌ Ошибка при обучении LSTM: {e}")
    raise

In [ ]:
# Оценка LSTM с вычислением ROUGE
import os
import pickle
import torch
from src.eval_lstm import evaluate_on_dataset
from src.lstm_model import LSTMModel

print("Загрузка модели LSTM для оценки...")

# Проверка файлов
model_path = "models/lstm_model.pth"
vocab_path = "models/vocab.pkl"

for path, name in [(vocab_path, "словаря"), (model_path, "модели")]:
    if not os.path.exists(path):
        raise FileNotFoundError(f"{name} не найден: {path}")

# Загрузка словаря
with open(vocab_path, "rb") as f:
    vocab = pickle.load(f)

pad_idx = vocab["<PAD>"]
vocab_size = len(vocab)

# Инициализация модели
model = LSTMModel(
    vocab_size=vocab_size,
    embedding_dim=128,
    hidden_dim=256,
    num_layers=2,
    dropout=0.3,
    pad_idx=pad_idx
).to(DEVICE)

# Загрузка весов
model.load(model_path, device=DEVICE)
model.eval()

print(f"✅ Модель загружена и переведена в режим eval. Устройство: {DEVICE}")

# Оценка
try:
    with torch.no_grad():  # Лучше, чем set_grad_enabled
        lstm_rouge = evaluate_on_dataset(
            model=model,
            split="val",
            batch_size=64,
            max_samples=500,
            device=DEVICE,
            max_gen_length=15,
            disable_tqdm=False
        )

    print("\n✅ Оценка завершена успешно.")
    # Предполагаем, что ключи: 'rouge1', 'rouge2', 'rougeL'
    print(f"ROUGE-1: {lstm_rouge.get('rouge1', 0):.4f}")
    print(f"ROUGE-2: {lstm_rouge.get('rouge2', 0):.4f}")
    print(f"ROUGE-L: {lstm_rouge.get('rougeL', 0):.4f}")

except KeyboardInterrupt:
    print("\n\n❌ Оценка прервана пользователем (Ctrl+C).")
    print("Попробуйте уменьшить max_samples или max_gen_length.")
except Exception as e:
    print(f"\n❌ Ошибка при оценке: {e}")
    import traceback
    traceback.print_exc()

In [ ]:
# Оценка предобученной модели DistilGPT-2
import torch

print("Запуск оценки DistilGPT-2...")

# Предупреждение о производительности
if DEVICE == "cpu":
    print("⚠️ Внимание: оценка DistilGPT-2 на CPU может занять 10+ минут. Рекомендуется использовать GPU.")

from src.eval_transformer_pipeline import main as evaluate_transformer

try:
    transformer_rouge = evaluate_transformer(
        model_name="distilgpt2",
        split="val",
        max_samples=500,
        max_length=30,
        device=DEVICE,
        batch_size=8
    )
    print("✅ Оценка DistilGPT-2 завершена.")

    # Вывод результатов
    print(f"ROUGE-1: {transformer_rouge['rouge1']:.4f}")
    print(f"ROUGE-2: {transformer_rouge['rouge2']:.4f}")
    print(f"ROUGE-L: {transformer_rouge['rougeL']:.4f}")

except Exception as e:
    print(f"❌ Ошибка при оценке DistilGPT-2: {e}")
    import traceback
    traceback.print_exc()
    raise

In [ ]:
# Сравнение метрик
print("Сравнение моделей по ROUGE-метрикам:")

# Проверка наличия данных
if 'lstm_rouge' not in globals():
    raise NameError("Переменная lstm_rouge не найдена. Выполните оценку LSTM.")
if 'transformer_rouge' not in globals():
    raise NameError("Переменная transformer_rouge не найдена. Выполните оценку DistilGPT-2.")

# Создание таблицы
results = {
    "Модель": ["LSTM", "DistilGPT-2"],
    "ROUGE-1": [lstm_rouge['rouge1'], transformer_rouge['rouge1']],
    "ROUGE-2": [lstm_rouge['rouge2'], transformer_rouge['rouge2']],
    "ROUGE-L": [lstm_rouge['rougeL'], transformer_rouge['rougeL']],
}

results_df = pd.DataFrame(results)

# Сортировка по ROUGE-1
results_df = results_df.sort_values(by="ROUGE-1", ascending=False).reset_index(drop=True)

# Вывод с подсветкой максимумов
display(results_df.round(4).style.highlight_max(color='lightgreen', subset=["ROUGE-1", "ROUGE-2", "ROUGE-L"]))

In [ ]:
# Примеры автодополнения от LSTM и DistilGPT-2
import torch

print("=== Примеры автодополнения ===\n")

# === LSTM ===
try:
    if 'vocab' not in globals():
        raise NameError("Словарь 'vocab' не найден. Выполните загрузку модели LSTM.")
    if 'model' not in globals():
        raise NameError("Модель LSTM не найдена. Выполните загрузку.")

    reverse_vocab = {idx: token for token, idx in vocab.items()}

    print("=== LSTM: Примеры автодополнения ===")
    with torch.no_grad():
        generate_examples(
            model=model,
            sample_texts=[
                "hello how are",
                "the weather today",
                "i want to tell",
                "in the bank there was"
            ],
            vocab=vocab,
            reverse_vocab=reverse_vocab,
            device=DEVICE,
            max_length=10
        )
except Exception as e:
    print(f"❌ Ошибка при генерации LSTM: {e}")

print()

# === DistilGPT-2 ===
try:
    print("=== DistilGPT-2: Примеры автодополнения ===")
    with torch.no_grad():
        generate_transformer_examples(
            model=transformer_model,
            tokenizer=tokenizer,
            sample_texts=[
                "hello how are",
                "the weather today",
                "i want to tell",
                "in the bank there was"
            ],
            max_gen_length=10
        )
except Exception as e:
    print(f"❌ Ошибка при генерации DistilGPT-2: {e}")

### 📊 Выводы

1. **Качество генерации**:
   - **DistilGPT-2** демонстрирует значительно более естественные и контекстуально корректные продолжения.
   - **LSTM** часто генерирует шаблонные или бессмысленные фразы, особенно при длинных продолжениях.

2. **ROUGE-метрики**:
   - DistilGPT-2 превосходит LSTM по всем метрикам (ROUGE-1, ROUGE-2, ROUGE-L), что говорит о лучшем соответствии с эталонными продолжениями.

3. **Вычислительные требования**:
   - LSTM: легковесная, быстро обучается, подходит для сред с ограниченными ресурсами.
   - DistilGPT-2: требует больше памяти и времени на инференс, но не требует обучения — только инференс.

4. **Поддержка и масштабируемость**:
   - DistilGPT-2 обучалась на огромном корпусе, что даёт ей преимущество в понимании языка.
   - LSTM требует большого качественного датасета и тонкой настройки.


In [ ]:
# Сохранение результатов
import os

print("Сохранение итоговых результатов...")

# Проверка наличия данных
if 'results_df' not in globals():
    raise NameError("Переменная results_df не найдена. Выполните сравнение моделей.")

# Создание директории
os.makedirs("results", exist_ok=True)

# Сохранение
try:
    results_df.to_csv("results/comparison_results.csv", index=False)
    print("✅ Итоговые результаты сохранены в results/comparison_results.csv")
except Exception as e:
    print(f"❌ Не удалось сохранить результаты: {e}")
    raise